In [152]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset

In [153]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [154]:
def encdec_accuracy(encoder, decoder, inf_gen, test_steps, device=DEVICE):
    correct = 0
    num_data = 0

    #grab a batch from the test loader
    for j in range(test_steps):
        examples, labels = next(inf_gen)
        embedding = encoder.forward(examples.to(device))
        outputs = decoder.forward(embedding.to(device))

        #for each output in the batch, check if the label is correct
        for i, output in enumerate(outputs):
            num_data += 1

            max_i = np.argmax(output.detach().cpu().numpy())
            if max_i == labels[i]:
                correct += 1

    acc = float(correct)/num_data
    
    return acc

In [155]:
def inf_loader_generator(dataloader):
    '''
    Generates a function that infinitely samples a dataloader
    '''
    while True:
        for x, y in dataloader:
            yield x, y

In [156]:
class mnist_Classifier(nn.Module):
    def __init__(self):
        super(mnist_Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 28, kernel_size=(5,5))
        self.conv2 = nn.Conv2d(28, 32, kernel_size=(5,5))
        #ok, i undersatnd this now. the output from conv2 is 32 channels. the remaining width and height after
        # two 5x5 filters with NO PADDING and STRIDE 1 (defaults) is 20x20. So 32 channels * 20 h * 20 w
        self.fc1 = nn.Linear(32*20*20, 16)
        self.fc2 = nn.Linear(16, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        # print(x.size()) --> torch.Size([16, 32, 20, 20])
        x = x.view(-1, 32*20*20)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        return torch.softmax(x,dim=1)

class EncoderCNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=28):
        super().__init__()
        self.hidden_size = hidden_size
        self.conv = nn.Conv2d(input_size, hidden_size, kernel_size=(5,5))
        self.output_shape = (hidden_size,24,24)

    def forward(self, x):
        x = self.conv(x)
        x = nn.ReLU()(x)
        return x
    
class DecoderCNN(nn.Module):
    def __init__(self, hidden_size=28):
        super().__init__()
        self.hidden_size = hidden_size
        self.conv = nn.Conv2d(hidden_size, 32, kernel_size=(5,5))
        self.fc1 = nn.Linear(32*20*20, 16)
        self.fc2 = nn.Linear(16, 10)

    def forward(self, x):
        x = self.conv(x)
        x = nn.ReLU()(x)
        x = x.view(-1, 32*20*20)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        return torch.softmax(x,dim=1)
    
class EncoderCNN2(nn.Module):
    def __init__(self, input_size=1, hidden_size=28):
        super().__init__()
        self.hidden_size = hidden_size
        self.conv1 = nn.Conv2d(input_size, hidden_size, kernel_size=(5,5))
        self.conv2 = nn.Conv2d(hidden_size, 32, kernel_size=(5,5))
        self.output_shape = (hidden_size,24,24)
        self.fc1 = nn.Linear(32*20*20, 16)
        self.fc2 = nn.Linear(16, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = x.view(-1, 32*20*20)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        return x
    
class DecoderCNN2(nn.Module):
    def __init__(self, hidden_size=28):
        super().__init__()
        self.hidden_size = hidden_size
        
    def forward(self, x):
        return torch.softmax(x,dim=1)

In [157]:
def compute_immediate_sensitivity(model, inp, loss) -> list:
    """Core. Computes immediate sensitivity"""

    # (1) first-order gradient (wrt parameters)
    first_order_grads = torch.autograd.grad(
        loss,
        model.parameters(),
        retain_graph=True,
        create_graph=True,
        # allow_unused=True
    )

    # (2) L2 norm of the gradient from (1)
    grad_l2_norm = torch.norm(torch.cat([x.view(-1) for x in first_order_grads]), p=2)

    # (3) Gradient (wrt inputs) of the L2 norm of the gradient from (2)
    sensitivity_vec = torch.autograd.grad(grad_l2_norm, inp, retain_graph=True)[0]

    # (4) L2 norm of (3) - "immediate sensitivity"
    # sensitivity = [torch.norm(v, p=2).item() for v in sensitivity_vec]
    sensitivity = torch.norm(
        sensitivity_vec.view(sensitivity_vec.shape[0], -1), p=2, dim=1
    )

    return sensitivity

In [158]:
def multiple_data_generators(n, dataset="mnist", folds=50, batch_sizes=16):
    if n > folds:
        raise ValueError(f"n {n} > {folds} folds")
    total = 60000
    fold_size = total//folds
    index = np.arange(total)
    np.random.shuffle(index)
    
    if np.array(batch_sizes).ndim == 0:
        batch_sizes = [batch_sizes] * n
    
    train_data_loaders = []
    test_data_loaders = []
    for i in range(n):
        batch_size = batch_sizes[i]
        if dataset == "mnist":
            mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
            mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())
        elif dataset == "fashionmnist":
            mnist_trainset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
            mnist_testset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())
        else:
            raise ValueError(f"dataset {dataset}")
            
        mnist_trainset = Subset(mnist_trainset, index[i * fold_size:(i+1) * fold_size])
        print(f"data slice [{index[i * fold_size]}, ..., {index[(i+1)* fold_size - 1]}]")
        #print(i * fold_size, (i+1) * fold_size)
        #print(len(mnist_trainset))
        #print(len(mnist_testset))
        max_trainsteps = len(mnist_trainset) // batch_size
        max_teststeps = len(mnist_testset) // batch_size
        #print(max_trainsteps)
        #print(max_teststeps)

        train_loader = DataLoader(mnist_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
        test_loader = DataLoader(mnist_testset, batch_size=batch_size, shuffle=True, drop_last=True)

        inf_train = inf_loader_generator(train_loader)
        inf_test = inf_loader_generator(test_loader)

        iters_per_epoch_train = len(mnist_trainset) // batch_size
        iters_per_epoch_test = len(mnist_testset) // batch_size
        
        train_data_loaders.append(inf_train)
        test_data_loaders.append(inf_test)
    
    return train_data_loaders, test_data_loaders
        

In [159]:
class MultipleDecoder:
    def __init__(self, n, hidden_size=28, device=DEVICE, alpha=20, epsilon=1e6, first_models=True, separate_encoders=False, separate_decoders=True, dataset="mnist", same_decoder_init=False, unbalanced=False):
        """
        Creates n decoders for single encoder
        """
        self.device = device
        self.n = n
        self.hidden_size = hidden_size
        self.alpha = alpha
        self.epsilon = epsilon
        self.dataset = dataset
        
        #if not first_models:
        #    print("Second Models")
        #    EncoderCNN = EncoderCNN2
        #    DecoderCNN = DecoderCNN2
        #    print(EncoderCNN)
        #    print(DecoderCNN)
        #else:
        #    EncoderCNN = EncoderCNN
        #    DecoderCNN = DecoderCNN
        
        self.separate_encoders = bool(separate_encoders)
        self.separate_decoders = bool(separate_decoders)
        if self.separate_encoders:
            self.encoders = []
            self.encoder_optimizers = []
            for i in range(self.n):
                self.encoders.append(EncoderCNN(1, self.hidden_size).to(self.device))
                self.encoder_optimizers.append(optim.Adam(self.encoders[i].parameters(),lr=0.001))
        else:
            self.encoder = EncoderCNN(1, self.hidden_size).to(self.device)
            self.encoder_optimizer = optim.Adam(self.encoder.parameters(),lr=0.001)
        
        if not self.separate_decoders:
            self.decoder = DecoderCNN(self.hidden_size).to(self.device)
            self.decoder_criterion = nn.CrossEntropyLoss()
            self.decoder_optimizer = optim.Adam(self.decoder.parameters(),lr=0.001)
        else:
            self.decoders = []
            self.decoder_criterions = []
            self.decoder_optimizers = []

            if same_decoder_init:
                decoder = DecoderCNN(self.hidden_size).to(self.device)
                for i in range(self.n):
                    if i == 0:
                        self.decoders.append(decoder)
                    else:
                        import copy
                        self.decoders.append(copy.deepcopy(decoder).to(self.device))
                    self.decoder_criterions.append(nn.CrossEntropyLoss())
                    self.decoder_optimizers.append(optim.Adam(self.decoders[i].parameters(),lr=0.001))
            else:
                for i in range(self.n):
                    self.decoders.append(DecoderCNN(self.hidden_size).to(self.device))
                    self.decoder_criterions.append(nn.CrossEntropyLoss())
                    self.decoder_optimizers.append(optim.Adam(self.decoders[i].parameters(),lr=0.001))
        
        if unbalanced:
            self.train_data_generators, self.test_data_generators = multiple_data_generators_unbalanced(self.n, batch_sizes=16, dataset=dataset)
        else:
            self.train_data_generators, self.test_data_generators = multiple_data_generators(self.n, batch_sizes=16, dataset=dataset)

    def train(self, epochs=20, train_steps_per_epoch=75, test_steps_per_epoch=50, final_test_steps=625, randomize_order=False, dp=True):
        self._train_init(epochs=epochs, train_steps_per_epoch=train_steps_per_epoch, test_steps_per_epoch=test_steps_per_epoch, randomize_order=randomize_order)
        
        import time
        start = time.time()
        elapsed_test_time = 0.0
        while True:
            if self.model_steps >= (self.train_steps_per_epoch*self.epochs):
                print("Finished Training")
                break
            if self.model_steps % self.train_steps_per_epoch == 0:
                print("Start of Epoch", (self.model_steps // self.train_steps_per_epoch))
  
            self._train_batch(dp=dp)

            if self.model_steps % self.train_steps_per_epoch == 0:
                train_losses = [round(10000 * np.mean(x))/10000 for x in self.train_losses]
                print(f"Average Train Loss:", train_losses)
                #for i in range(self.n):
                #    print(f"Average Train Loss for {i}:", np.mean(self.train_losses[i]))
                
                if test_steps_per_epoch:
                    start_test = time.time()
                    for i in range(self.n):
                        if self.separate_encoders:
                            encoder = self.encoders[i]
                        else:
                            encoder = self.encoder
                        if self.separate_decoders:
                            decoder = self.decoders[i]
                        else:
                            decoder = self.decoder
                        self.test_accs[i].append(encdec_accuracy(encoder, decoder, self.test_data_generators[i], self.test_steps_per_epoch, device=self.device))
                
                    test_accs = [x[-1] for x in self.test_accs]
                    print(f"Test Accuracy:", test_accs)
                    end_test = time.time()
                    elapsed_test_time += end_test - start_test
                #for i in range(self.n):
                #    print(f"Test Accuracy {i}:", self.test_accs[i][-1])
        
        end = time.time()
        elapsed = end - start
        self.training_time = elapsed - elapsed_test_time
        print(f"Elapsed Test Time:", elapsed_test_time)
        print(f"Elapsed Training Time:", self.training_time)
        for i in range(self.n):
            if self.separate_encoders:
                encoder = self.encoders[i]
            else:
                encoder = self.encoder
            if self.separate_decoders:
                decoder = self.decoders[i]
            else:
                decoder = self.decoder
            self.test_accs[i].append(encdec_accuracy(encoder, decoder, self.test_data_generators[i], final_test_steps, device=self.device))
            
        test_accs = [x[-1] for x in self.test_accs]
        print(f"Final Test Accuracy:", test_accs)
    
    def _train_init(self, epochs=20, train_steps_per_epoch=375, test_steps_per_epoch=100, randomize_order=False):
        self.epochs = epochs
        self.model_steps = 0
        if self.alpha is None or self.epsilon is None:
            self.epsilon_iter = None
        else:
            self.epsilon_iter = self.epsilon / self.epochs
        
        self.train_steps_per_epoch = train_steps_per_epoch
        self.test_steps_per_epoch = test_steps_per_epoch
        self.randomize_order = randomize_order
        
        # plotting criteria
        self.train_losses = [[] for x in range(self.n)]
        self.test_accs = [[] for x in range(self.n)]        

    def _train_batch(self, dp=True):
        index = list(range(self.n))
        if self.randomize_order:
            np.random.shuffle(index)
        
        for i in index:
            if dp:
                self._train_decoder_batch_dp(i)
            else:
                self._train_decoder_batch_no_dp(i)
        
        self.model_steps += 1
        
    def _train_decoder_batch_no_dp(self, i):
        """
        Train decoder i on the next batch without differential privacy
        """
        if self.separate_encoders:
            encoder = self.encoders[i]
            encoder_optimizer = self.encoder_optimizers[i]
        else:
            encoder = self.encoder
            encoder_optimizer = self.encoder_optimizer
        if self.separate_decoders:
            decoder = self.decoders[i]
            decoder_optimizer = self.decoder_optimizers[i]
            decoder_criterion = self.decoder_criterions[i]
        else:
            decoder = self.decoder
            decoder_optimizer = self.decoder_optimizer
            decoder_criterion = self.decoder_criterion
        
        x_batch_train, y_batch_train = (x.to(self.device) for x in next(self.train_data_generators[i]))
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        # compute loss
        embedding = encoder.forward(x_batch_train)
        outputs = decoder.forward(embedding)
        loss = decoder_criterion(outputs, y_batch_train)

        loss.backward()
        self.train_losses[i].append(loss.item())

        # perform the a gradient step
        encoder_optimizer.step()
        decoder_optimizer.step()
        
    def _train_decoder_batch_dp(self, i):
        """
        Train decoder i on the next batch using differential privacy
        """
        if self.separate_encoders:
            encoder = self.encoders[i]
            encoder_optimizer = self.encoder_optimizers[i]
        else:
            encoder = self.encoder
            encoder_optimizer = self.encoder_optimizer
        if self.separate_decoders:
            decoder = self.decoders[i]
            decoder_optimizer = self.decoder_optimizers[i]
            decoder_criterion = self.decoder_criterions[i]
        else:
            decoder = self.decoder
            decoder_optimizer = self.decoder_optimizer
            decoder_criterion = self.decoder_criterion
        
        x_batch_train, y_batch_train = (x.to(self.device) for x in next(self.train_data_generators[i]))
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        # Need grad on input for sensitivity; not sure if cloning is needed
        x_batch_train = torch.autograd.Variable(torch.clone(x_batch_train).to(self.device), requires_grad=True)
        
        # compute loss
        embedding = encoder.forward(x_batch_train)
        outputs = decoder.forward(embedding)
        loss = decoder_criterion(outputs, y_batch_train)
        
        batch_sensitivities = compute_immediate_sensitivity(
            encoder, x_batch_train, loss
        )
        batch_sensitivity = torch.max(batch_sensitivities) / len(x_batch_train)
        
        loss.backward()
        # step 4. compute noise
        # this is the scale of the Gaussian noise to be added to the batch gradient
        sigma = torch.sqrt(
            (batch_sensitivity ** 2 * self.alpha) / (2 * self.epsilon_iter)
        )
        
        # logging
        #x = loss.item()
        #if np.isnan(x):
        #    log.debug(
        #        f"Epoch (before adj): {epoch}; batch_idx: {batch_idx} loss: {loss}"
        #    )
        #    x = 0
        #    log.debug(
        #        f"    epoch (adfter adj): {epoch}; batch_idx: {batch_idx} loss: {loss}"
        #    )
        #train_loss += x
        
        self.train_losses[i].append(loss.item())
        
        # step 5. update gradients with computed sensitivities
        with torch.no_grad():
            for p in encoder.parameters():
                p.grad += sigma * torch.randn(1).to(self.device)
        
        # perform the a gradient step
        encoder_optimizer.step()
        decoder_optimizer.step()

    def fine_tune(self, dp=False, **kwargs):
        """
        Used to fine tune individual encoder / decoder pairs AFTER primary collective training
        """
        import copy
        self.separate_encoders = True
        if self.separate_encoders:
            self.encoders = []
            self.encoder_optimizers = []
            for i in range(self.n):
                self.encoders.append(copy.deepcopy(self.encoder).to(self.device))
                self.encoder_optimizers.append(optim.Adam(self.encoders[i].parameters(),lr=0.001))

        self.training_time_dp = self.training_time
        self.test_accs_dp = self.test_accs
        self.train(dp=dp, **kwargs)
        
        self.training_time += self.training_time_dp

In [160]:
m = MultipleDecoder(10, epsilon=1e6)
m.train(dp=True, epochs=3)
m.fine_tune(epochs=1)
test_accs = [x[-1] for x in m.test_accs]
training_times = m.training_time

print(f"Average training time", np.mean(training_times))
print(f"Average test accuracy", np.mean(test_accs))
print(f"Median test accuracy", np.median(test_accs))
print(f"Standard deviation test accuracy", np.std(test_accs))

data slice [19821, ..., 22599]
data slice [20777, ..., 46303]
data slice [4275, ..., 46837]
data slice [2075, ..., 54875]
data slice [20274, ..., 44860]
data slice [21638, ..., 44344]
data slice [45998, ..., 482]
data slice [8153, ..., 48410]
data slice [56503, ..., 50922]
data slice [6358, ..., 26813]
Start of Epoch 0
Average Train Loss: [2.0524, 2.1395, 2.0987, 2.1257, 2.0831, 2.0851, 2.0488, 2.2103, 2.1478, 2.1269]
Test Accuracy: [0.53, 0.385, 0.5175, 0.4675, 0.54625, 0.50375, 0.52875, 0.195, 0.41625, 0.3875]
Start of Epoch 1
Average Train Loss: [1.9913, 2.0939, 2.0195, 2.0763, 1.998, 2.0221, 1.9783, 2.1618, 2.0415, 2.0919]
Test Accuracy: [0.5125, 0.49875, 0.54875, 0.49625, 0.5675, 0.57625, 0.59625, 0.35375, 0.55625, 0.48125]
Start of Epoch 2
Average Train Loss: [1.9587, 2.0343, 1.9868, 2.0568, 1.9561, 1.9982, 1.9265, 2.1138, 1.984, 2.0396]
Test Accuracy: [0.555, 0.58125, 0.54125, 0.45125, 0.6, 0.55125, 0.6525, 0.37375, 0.62125, 0.58375]
Finished Training
Elapsed Test Time: 4.734881

In [142]:
m = MultipleDecoder(10, epsilon=1e6)
m.train(dp=True, epochs=15)
m.fine_tune(epochs=5)
test_accs = [x[-1] for x in m.test_accs]
training_times = m.training_time

print(f"Average training time", np.mean(training_times))
print(f"Average test accuracy", np.mean(test_accs))
print(f"Median test accuracy", np.median(test_accs))
print(f"Standard deviation test accuracy", np.std(test_accs))

data slice [23223, ..., 17206]
data slice [55630, ..., 25122]
data slice [1601, ..., 34293]
data slice [33619, ..., 58677]
data slice [5187, ..., 27918]
data slice [12793, ..., 29042]
data slice [9856, ..., 47779]
data slice [21065, ..., 8449]
data slice [40734, ..., 41870]
data slice [22216, ..., 2627]
Start of Epoch 0
Average Train Loss: [2.058, 2.0857, 2.0596, 2.0921, 2.16, 2.0385, 2.1345, 2.1231, 2.057, 2.1255]
Test Accuracy: [0.5475, 0.5425, 0.61125, 0.485, 0.40625, 0.4575, 0.39, 0.345, 0.4875, 0.50625]
Start of Epoch 1
Average Train Loss: [1.9657, 2.004, 1.9185, 1.9982, 2.0823, 1.9843, 2.0652, 2.0625, 2.0216, 2.0148]
Test Accuracy: [0.64625, 0.5575, 0.69625, 0.53625, 0.4475, 0.50625, 0.4825, 0.4525, 0.47125, 0.56]
Start of Epoch 2
Average Train Loss: [1.91, 1.9234, 1.8512, 1.9568, 2.0281, 1.9427, 2.0291, 2.0215, 2.0013, 1.9719]
Test Accuracy: [0.68625, 0.66375, 0.74875, 0.56375, 0.5125, 0.58375, 0.5075, 0.45625, 0.52375, 0.55625]
Start of Epoch 3
Average Train Loss: [1.8677, 1.86

In [118]:
def multiple_data_generators_unbalanced(n, dataset="mnist", folds=50, batch_sizes=16):
    if n != 10:
        raise NotImplementedError("n must currently be 10")
    if batch_sizes != 16:
        raise NotImplementedError("batch_sizes must currently be 16")
    if folds != 50:
        raise NotImplementedError("folds must currently be 50")
    if n > folds:
        raise ValueError(f"n {n} > {folds} folds")
    total = 60000
    if np.array(batch_sizes).ndim == 0:
        batch_sizes = [batch_sizes] * n
        
    main_total = 660 # primary class
    off_total = 60  # other classes
    
    batch_size = 16
    if dataset == "mnist":
        trainset = datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
    elif dataset == "fashionmnist":
        trainset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
    else:
        raise ValueError(f"dataset {dataset}")
    dataloader = DataLoader(trainset, batch_size=16, shuffle=False, drop_last=True)
    ys = []
    for x, y in dataloader:
        ys.append(y)
    y = np.hstack(ys)

    classes = []
    n = 10  # number of classes
    for i in range(n):
        classes.append(np.where(y == i)[0])
        np.random.shuffle(classes[i])

    indexes = [[] for x in range(n)]
    for i in range(n):  # for each class
        c = classes[i]
        start = 0
        for j in range(n):  # for each organization
            index = indexes[j]
            if i == j:  # main total
                index.extend(c[start:start+main_total])
                start += main_total
            else:  # off total
                index.extend(c[start:start+off_total])
                start += off_total

    for i in range(n):
        print(np.bincount(y[indexes[i]]))

    train_data_loaders = []
    test_data_loaders = []
    for i in range(n):
        batch_size = batch_sizes[i]
        if dataset == "mnist":
            mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
            mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())
        elif dataset == "fashionmnist":
            mnist_trainset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
            mnist_testset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())
        else:
            raise ValueError(f"dataset {dataset}")
            
        mnist_trainset = Subset(mnist_trainset, indexes[i])
        #print(i * fold_size, (i+1) * fold_size)
        print(len(mnist_trainset))
        print(len(mnist_testset))
        max_trainsteps = len(mnist_trainset) // batch_size
        max_teststeps = len(mnist_testset) // batch_size
        #print(max_trainsteps)
        #print(max_teststeps)

        train_loader = DataLoader(mnist_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
        test_loader = DataLoader(mnist_testset, batch_size=batch_size, shuffle=True, drop_last=True)

        inf_train = inf_loader_generator(train_loader)
        inf_test = inf_loader_generator(test_loader)

        iters_per_epoch_train = len(mnist_trainset) // batch_size
        iters_per_epoch_test = len(mnist_testset) // batch_size
        
        train_data_loaders.append(inf_train)
        test_data_loaders.append(inf_test)
    
    return train_data_loaders, test_data_loaders
        

In [10]:
test_accs = []
training_times = []
for i in range(2):
    m = MultipleDecoder(10, separate_encoders=True)
    m.train(dp=False, epochs=20)
    test_accs.extend([x[-1] for x in m.test_accs])
    training_times.append(m.training_time)
    
print(f"Average training time", np.mean(training_times))
print(f"Average test accuracy", np.mean(test_accs))
print(f"Median test accuracy", np.median(test_accs))
print(f"Standard deviation test accuracy", np.std(test_accs))

data slice [2445, ..., 52099]
data slice [45448, ..., 13145]
data slice [57776, ..., 31567]
data slice [55419, ..., 10951]
data slice [511, ..., 23039]
data slice [8511, ..., 39363]
data slice [17716, ..., 47010]
data slice [19691, ..., 51794]
data slice [12542, ..., 17397]
data slice [41737, ..., 36102]
Start of Epoch 0
Average Train Loss: [2.1212, 2.1458, 2.0703, 2.1274, 2.1555, 2.0967, 2.0824, 2.0741, 2.1347, 2.1517]
Test Accuracy: [0.58625, 0.40375, 0.5225, 0.46125, 0.3625, 0.43375, 0.5475, 0.48875, 0.37, 0.445]
Start of Epoch 1
Average Train Loss: [2.0071, 2.0844, 1.9851, 2.0296, 2.0947, 1.9997, 2.0054, 2.0087, 2.0125, 2.0344]
Test Accuracy: [0.605, 0.36875, 0.59, 0.52375, 0.39625, 0.48875, 0.575, 0.47, 0.61, 0.55375]
Start of Epoch 2
Average Train Loss: [1.9563, 2.0611, 1.95, 1.9893, 2.0453, 1.9592, 1.9561, 1.9701, 1.9324, 1.9794]
Test Accuracy: [0.61, 0.4325, 0.56375, 0.54375, 0.4875, 0.54125, 0.64, 0.5625, 0.6825, 0.6225]
Start of Epoch 3
Average Train Loss: [1.9229, 2.0412, 1.

In [21]:
test_accs_dp2 = []
training_times_dp2 = []
for i in range(10):
    m = MultipleDecoder(2, separate_encoders=False)
    m.train(dp=True, epochs=10)
    m.fine_tune(epochs=10)
    test_accs_dp2.extend([x[-1] for x in m.test_accs])
    training_times_dp2.append(m.training_time)

print(f"Average training time", np.mean(training_times_dp2))
print(f"Average test accuracy", np.mean(test_accs_dp2))
print(f"Median test accuracy", np.median(test_accs_dp2))
print(f"Standard deviation test accuracy", np.std(test_accs_dp2))

data slice [39645, ..., 3401]
data slice [50470, ..., 8563]
Start of Epoch 0
Average Train Loss: [2.0409, 2.0566]
Test Accuracy: [0.5225, 0.645]
Start of Epoch 1
Average Train Loss: [1.9675, 1.9201]
Test Accuracy: [0.5625, 0.71375]
Start of Epoch 2
Average Train Loss: [1.9364, 1.8672]
Test Accuracy: [0.60625, 0.7025]
Start of Epoch 3
Average Train Loss: [1.9178, 1.831]
Test Accuracy: [0.55125, 0.7175]
Start of Epoch 4
Average Train Loss: [1.9004, 1.8053]
Test Accuracy: [0.675, 0.71625]
Start of Epoch 5
Average Train Loss: [1.8781, 1.786]
Test Accuracy: [0.7, 0.74625]
Start of Epoch 6
Average Train Loss: [1.8523, 1.7712]
Test Accuracy: [0.74625, 0.74875]
Start of Epoch 7
Average Train Loss: [1.8307, 1.7591]
Test Accuracy: [0.76, 0.7625]
Start of Epoch 8
Average Train Loss: [1.8128, 1.7499]
Test Accuracy: [0.75625, 0.7625]
Start of Epoch 9
Average Train Loss: [1.7984, 1.7418]
Test Accuracy: [0.7675, 0.76625]
Finished Training
Elapsed Test Time: 3.144005537033081
Elapsed Training Time: 11

In [20]:
test_accs_dp5 = []
training_times_dp5 = []
for i in range(4):
    m = MultipleDecoder(5, separate_encoders=False)
    m.train(dp=True, epochs=15)
    m.fine_tune(epochs=5)
    test_accs_dp5.extend([x[-1] for x in m.test_accs])
    training_times_dp5.append(m.training_time)

print(f"Average training time", np.mean(training_times_dp5))
print(f"Average test accuracy", np.mean(test_accs_dp5))
print(f"Median test accuracy", np.median(test_accs_dp5))
print(f"Standard deviation test accuracy", np.std(test_accs_dp5))

data slice [16892, ..., 15936]
data slice [54144, ..., 10471]
data slice [11877, ..., 27188]
data slice [47255, ..., 25254]
data slice [31301, ..., 54184]
Start of Epoch 0
Average Train Loss: [2.0284, 2.1329, 2.2076, 2.0991, 2.0288]
Test Accuracy: [0.515, 0.3525, 0.2925, 0.5975, 0.61375]
Start of Epoch 1
Average Train Loss: [1.9313, 2.085, 2.1308, 1.98, 1.9359]
Test Accuracy: [0.65375, 0.38875, 0.39625, 0.605, 0.65125]
Start of Epoch 2
Average Train Loss: [1.8866, 2.0641, 2.0904, 1.9235, 1.8929]
Test Accuracy: [0.63625, 0.3875, 0.49125, 0.6325, 0.69875]
Start of Epoch 3
Average Train Loss: [1.858, 2.0497, 2.0654, 1.8906, 1.8507]
Test Accuracy: [0.67375, 0.375, 0.5025, 0.65125, 0.7675]
Start of Epoch 4
Average Train Loss: [1.8273, 2.0391, 2.0472, 1.869, 1.8208]
Test Accuracy: [0.7775, 0.34375, 0.50875, 0.64125, 0.7175]
Start of Epoch 5
Average Train Loss: [1.801, 2.0312, 2.0328, 1.8542, 1.8012]
Test Accuracy: [0.8, 0.35, 0.54375, 0.66375, 0.7475]
Start of Epoch 6
Average Train Loss: [1.

In [19]:
test_accs_dp10 = []
training_times_dp10 = []
for i in range(2):
    m = MultipleDecoder(10, separate_encoders=False)
    m.train(dp=True, epochs=15)
    m.fine_tune(epochs=5)
    test_accs_dp10.extend([x[-1] for x in m.test_accs])
    training_times_dp10.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp10))
print(f"Average test accuracy", np.mean(test_accs_dp10))
print(f"Median test accuracy", np.median(test_accs_dp10))
print(f"Standard deviation test accuracy", np.std(test_accs_dp10))

data slice [29957, ..., 33986]
data slice [46014, ..., 53856]
data slice [1204, ..., 8952]
data slice [47374, ..., 57370]
data slice [48104, ..., 46459]
data slice [33418, ..., 14436]
data slice [30013, ..., 36606]
data slice [50100, ..., 52116]
data slice [436, ..., 6761]
data slice [47780, ..., 23722]
Start of Epoch 0
Average Train Loss: [2.1285, 2.0171, 2.1045, 2.0914, 2.0117, 2.0043, 2.105, 2.0332, 2.1361, 1.9452]
Test Accuracy: [0.4875, 0.575, 0.445, 0.4975, 0.5825, 0.5425, 0.4575, 0.48375, 0.4375, 0.6825]
Start of Epoch 1
Average Train Loss: [2.0022, 1.9106, 2.0418, 1.9646, 1.9368, 1.9455, 2.0147, 1.9917, 2.0598, 1.8188]
Test Accuracy: [0.6725, 0.70125, 0.5575, 0.6675, 0.6075, 0.565, 0.52875, 0.52625, 0.5025, 0.8375]
Start of Epoch 2
Average Train Loss: [1.9413, 1.8514, 1.9965, 1.8869, 1.9018, 1.9221, 1.9677, 1.974, 2.0048, 1.7556]
Test Accuracy: [0.65375, 0.74, 0.55875, 0.7225, 0.65125, 0.53625, 0.65375, 0.50125, 0.55125, 0.8275]
Start of Epoch 3
Average Train Loss: [1.9028, 1.8

In [22]:
fashion_test_accs = []
fashion_training_times = []
for i in range(2):
    m = MultipleDecoder(10, separate_encoders=True, dataset="fashionmnist")
    m.train(dp=False, epochs=20)
    fashion_test_accs.extend([x[-1] for x in m.test_accs])
    fashion_training_times.append(m.training_time)

print(f"Average training time", np.mean(fashion_training_times))
print(f"Average test accuracy", np.mean(fashion_test_accs))
print(f"Median test accuracy", np.median(fashion_test_accs))
print(f"Standard deviation test accuracy", np.std(fashion_test_accs))

data slice [3727, ..., 54019]
data slice [13485, ..., 25627]
data slice [23935, ..., 6722]
data slice [58849, ..., 54073]
data slice [25174, ..., 25905]
data slice [45770, ..., 29955]
data slice [32628, ..., 50356]
data slice [49300, ..., 57944]
data slice [57056, ..., 59654]
data slice [57894, ..., 39400]
Start of Epoch 0
Average Train Loss: [2.2058, 2.0464, 2.15, 2.1601, 2.1967, 2.2654, 2.141, 2.0973, 2.1315, 2.1118]
Test Accuracy: [0.325, 0.50875, 0.37875, 0.3425, 0.45, 0.19625, 0.355, 0.43625, 0.4375, 0.5]
Start of Epoch 1
Average Train Loss: [2.1382, 1.9851, 2.0833, 2.1325, 2.1163, 2.227, 2.117, 2.0549, 2.0889, 2.0334]
Test Accuracy: [0.3625, 0.52625, 0.47125, 0.37625, 0.4725, 0.22125, 0.3425, 0.5175, 0.44875, 0.575]
Start of Epoch 2
Average Train Loss: [2.1075, 1.9608, 2.0507, 2.1195, 2.0842, 2.2062, 2.1063, 2.0227, 2.0691, 1.9839]
Test Accuracy: [0.43875, 0.5275, 0.48875, 0.37375, 0.46375, 0.2325, 0.38875, 0.48375, 0.44125, 0.62]
Start of Epoch 3
Average Train Loss: [2.0902, 1.9

In [23]:
fashion_test_accs_dp10 = []
fashion_training_times_dp10 = []
for i in range(2):
    m = MultipleDecoder(10, separate_encoders=False, dataset="fashionmnist")
    m.train(dp=True, epochs=15)
    m.fine_tune(epochs=5)
    fashion_test_accs_dp10.extend([x[-1] for x in m.test_accs])
    fashion_training_times_dp10.append(m.training_time)
print(f"Average training time", np.mean(fashion_training_times_dp10))
print(f"Average test accuracy", np.mean(fashion_test_accs_dp10))
print(f"Median test accuracy", np.median(fashion_test_accs_dp10))
print(f"Standard deviation test accuracy", np.std(fashion_test_accs_dp10))

data slice [22711, ..., 8166]
data slice [8128, ..., 25808]
data slice [52745, ..., 20581]
data slice [55604, ..., 39027]
data slice [34421, ..., 41712]
data slice [10042, ..., 51753]
data slice [3329, ..., 15414]
data slice [28241, ..., 14988]
data slice [10570, ..., 28244]
data slice [4254, ..., 2750]
Start of Epoch 0
Average Train Loss: [2.1193, 2.1229, 2.1559, 2.1155, 2.1719, 2.1442, 2.1362, 2.1769, 2.0395, 2.1756]
Test Accuracy: [0.395, 0.34875, 0.3725, 0.45625, 0.33375, 0.45875, 0.3175, 0.44625, 0.6625, 0.36375]
Start of Epoch 1
Average Train Loss: [2.054, 2.0764, 2.1024, 2.0344, 2.1312, 2.0734, 2.1064, 2.0821, 1.9319, 2.0896]
Test Accuracy: [0.52125, 0.495, 0.42625, 0.5075, 0.4025, 0.4375, 0.3475, 0.515, 0.6775, 0.45875]
Start of Epoch 2
Average Train Loss: [2.0115, 2.0477, 2.0755, 1.9992, 2.0894, 2.0418, 2.0955, 2.0329, 1.8871, 2.0498]
Test Accuracy: [0.50875, 0.5075, 0.44375, 0.4925, 0.4, 0.475, 0.39125, 0.50625, 0.67375, 0.4725]
Start of Epoch 3
Average Train Loss: [1.9823, 2

In [24]:
fashion_test_accs_dp5 = []
fashion_training_times_dp5 = []
for i in range(4):
    m = MultipleDecoder(5, separate_encoders=False, dataset="fashionmnist")
    m.train(dp=True, epochs=15)
    m.fine_tune(epochs=5)
    fashion_test_accs_dp5.extend([x[-1] for x in m.test_accs])
    fashion_training_times_dp5.append(m.training_time)
print(f"Average training time", np.mean(fashion_training_times_dp5))
print(f"Average test accuracy", np.mean(fashion_test_accs_dp5))
print(f"Median test accuracy", np.median(fashion_test_accs_dp5))
print(f"Standard deviation test accuracy", np.std(fashion_test_accs_dp5))

data slice [33731, ..., 32461]
data slice [51560, ..., 36488]
data slice [1378, ..., 43316]
data slice [9374, ..., 40701]
data slice [12715, ..., 48920]
Start of Epoch 0
Average Train Loss: [2.1512, 2.1927, 2.1242, 2.1877, 2.0491]
Test Accuracy: [0.395, 0.29375, 0.50875, 0.3775, 0.47875]
Start of Epoch 1
Average Train Loss: [2.1048, 2.153, 2.0473, 2.1314, 1.9951]
Test Accuracy: [0.33625, 0.3175, 0.4375, 0.42375, 0.47375]
Start of Epoch 2
Average Train Loss: [2.0838, 2.1173, 2.0096, 2.1007, 1.9644]
Test Accuracy: [0.43, 0.39125, 0.52375, 0.42125, 0.505]
Start of Epoch 3
Average Train Loss: [2.0676, 2.0757, 1.9877, 2.0834, 1.9488]
Test Accuracy: [0.41375, 0.555, 0.50875, 0.41375, 0.545]
Start of Epoch 4
Average Train Loss: [2.0566, 2.0384, 1.973, 2.0689, 1.9351]
Test Accuracy: [0.385, 0.6075, 0.51625, 0.4725, 0.59875]
Start of Epoch 5
Average Train Loss: [2.049, 2.0109, 1.9624, 2.0586, 1.914]
Test Accuracy: [0.41625, 0.59125, 0.54375, 0.47875, 0.62375]
Start of Epoch 6
Average Train Loss

In [25]:
fashion_test_accs_dp2 = []
fashion_training_times_dp2 = []
for i in range(10):
    m = MultipleDecoder(2, separate_encoders=False, dataset="fashionmnist")
    m.train(dp=True, epochs=10)
    m.fine_tune(epochs=10)
    fashion_test_accs_dp2.extend([x[-1] for x in m.test_accs])
    fashion_training_times_dp2.append(m.training_time)
print(f"Average training time", np.mean(fashion_training_times_dp2))
print(f"Average test accuracy", np.mean(fashion_test_accs_dp2))
print(f"Median test accuracy", np.median(fashion_test_accs_dp2))
print(f"Standard deviation test accuracy", np.std(fashion_test_accs_dp2))

data slice [50038, ..., 55242]
data slice [16154, ..., 30514]
Start of Epoch 0
Average Train Loss: [2.0735, 2.1326]
Test Accuracy: [0.3425, 0.3825]
Start of Epoch 1
Average Train Loss: [2.0239, 2.0757]
Test Accuracy: [0.4875, 0.4825]
Start of Epoch 2
Average Train Loss: [1.9786, 2.0388]
Test Accuracy: [0.635, 0.525]
Start of Epoch 3
Average Train Loss: [1.9463, 2.016]
Test Accuracy: [0.61375, 0.495]
Start of Epoch 4
Average Train Loss: [1.924, 2.001]
Test Accuracy: [0.62875, 0.51875]
Start of Epoch 5
Average Train Loss: [1.9088, 1.989]
Test Accuracy: [0.59875, 0.4925]
Start of Epoch 6
Average Train Loss: [1.8966, 1.9763]
Test Accuracy: [0.6725, 0.5075]
Start of Epoch 7
Average Train Loss: [1.8865, 1.9671]
Test Accuracy: [0.62375, 0.54]
Start of Epoch 8
Average Train Loss: [1.8781, 1.9585]
Test Accuracy: [0.64125, 0.51625]
Start of Epoch 9
Average Train Loss: [1.8707, 1.9507]
Test Accuracy: [0.6325, 0.60125]
Finished Training
Elapsed Test Time: 3.166701316833496
Elapsed Training Time: 1

In [26]:
fashion_test_accs_dp10_same = []
fashion_training_times_dp10_same = []
for i in range(2):
    m = MultipleDecoder(10, separate_encoders=False, dataset="fashionmnist", same_decoder_init=True)
    m.train(dp=True, epochs=15)
    m.fine_tune(epochs=5)
    fashion_test_accs_dp10_same.extend([x[-1] for x in m.test_accs])
    fashion_training_times_dp10_same.append(m.training_time)
print(f"Average training time", np.mean(fashion_training_times_dp10_same))
print(f"Average test accuracy", np.mean(fashion_test_accs_dp10_same))
print(f"Median test accuracy", np.median(fashion_test_accs_dp10_same))
print(f"Standard deviation test accuracy", np.std(fashion_test_accs_dp10_same))

data slice [43432, ..., 24959]
data slice [33567, ..., 7406]
data slice [58272, ..., 2226]
data slice [25471, ..., 39206]
data slice [16835, ..., 9715]
data slice [2042, ..., 42968]
data slice [7904, ..., 34047]
data slice [13484, ..., 2886]
data slice [21330, ..., 8317]
data slice [29160, ..., 35670]
Start of Epoch 0
Average Train Loss: [2.0855, 2.2079, 2.1509, 2.0917, 2.1869, 2.121, 2.1748, 2.1884, 2.1953, 2.1361]
Test Accuracy: [0.44875, 0.37, 0.31875, 0.455, 0.38125, 0.35875, 0.315, 0.33125, 0.33875, 0.3575]
Start of Epoch 1
Average Train Loss: [2.045, 2.1494, 2.1129, 2.041, 2.1331, 2.0599, 2.1258, 2.0929, 2.1233, 2.0939]
Test Accuracy: [0.44375, 0.36625, 0.33875, 0.46625, 0.385, 0.46875, 0.39625, 0.54375, 0.415, 0.43375]
Start of Epoch 2
Average Train Loss: [2.0286, 2.127, 2.0969, 2.0111, 2.1059, 2.025, 2.0997, 2.0392, 2.0786, 2.048]
Test Accuracy: [0.43375, 0.40375, 0.37, 0.51125, 0.39125, 0.50375, 0.375, 0.535, 0.54125, 0.53875]
Start of Epoch 3
Average Train Loss: [2.0185, 2.11

In [27]:
fashion_test_accs_dp5_same = []
fashion_training_times_dp5_same = []
for i in range(4):
    m = MultipleDecoder(5, separate_encoders=False, dataset="fashionmnist", same_decoder_init=True)
    m.train(dp=True, epochs=15)
    m.fine_tune(epochs=5)
    fashion_test_accs_dp5_same.extend([x[-1] for x in m.test_accs])
    fashion_training_times_dp5_same.append(m.training_time)
print(f"Average training time", np.mean(fashion_training_times_dp5_same))
print(f"Average test accuracy", np.mean(fashion_test_accs_dp5_same))
print(f"Median test accuracy", np.median(fashion_test_accs_dp5_same))
print(f"Standard deviation test accuracy", np.std(fashion_test_accs_dp5_same))

data slice [29336, ..., 58232]
data slice [47210, ..., 14346]
data slice [40770, ..., 5904]
data slice [50822, ..., 53889]
data slice [50191, ..., 42718]
Start of Epoch 0
Average Train Loss: [2.1445, 2.1885, 2.1421, 2.1372, 2.1485]
Test Accuracy: [0.35625, 0.35625, 0.41375, 0.4125, 0.4475]
Start of Epoch 1
Average Train Loss: [2.0942, 2.1278, 2.1076, 2.0785, 2.0697]
Test Accuracy: [0.4625, 0.36, 0.37875, 0.4125, 0.46]
Start of Epoch 2
Average Train Loss: [2.0735, 2.072, 2.0867, 2.0534, 2.0293]
Test Accuracy: [0.47875, 0.515, 0.37875, 0.43875, 0.5325]
Start of Epoch 3
Average Train Loss: [2.0615, 2.0395, 2.0731, 2.0326, 2.0059]
Test Accuracy: [0.4425, 0.4925, 0.39, 0.55, 0.51875]
Start of Epoch 4
Average Train Loss: [2.0519, 2.018, 2.0649, 2.0033, 1.9894]
Test Accuracy: [0.4625, 0.53, 0.42375, 0.6125, 0.52125]
Start of Epoch 5
Average Train Loss: [2.0442, 2.0014, 2.0594, 1.98, 1.9757]
Test Accuracy: [0.45, 0.54875, 0.37125, 0.58125, 0.60875]
Start of Epoch 6
Average Train Loss: [2.038, 

In [28]:
test_accs_unbalanced = []
training_times_unbalanced = []
for i in range(2):
    m = MultipleDecoder(10, separate_encoders=True, dataset="mnist", unbalanced=True)
    m.train(dp=False, epochs=20)
    test_accs_unbalanced.extend([x[-1] for x in m.test_accs])
    training_times_unbalanced.append(m.training_time)

print(f"Average training time", np.mean(training_times_unbalanced))
print(f"Average test accuracy", np.mean(test_accs_unbalanced))
print(f"Median test accuracy", np.median(test_accs_unbalanced))
print(f"Standard deviation test accuracy", np.std(test_accs_unbalanced))

[660  60  60  60  60  60  60  60  60  60]
[ 60 660  60  60  60  60  60  60  60  60]
[ 60  60 660  60  60  60  60  60  60  60]
[ 60  60  60 660  60  60  60  60  60  60]
[ 60  60  60  60 660  60  60  60  60  60]
[ 60  60  60  60  60 660  60  60  60  60]
[ 60  60  60  60  60  60 660  60  60  60]
[ 60  60  60  60  60  60  60 660  60  60]
[ 60  60  60  60  60  60  60  60 660  60]
[ 60  60  60  60  60  60  60  60  60 660]
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
Start of Epoch 0
Average Train Loss: [2.2631, 1.9373, 2.2726, 1.9281, 1.9392, 1.926, 1.9367, 2.3029, 1.9325, 1.9213]
Test Accuracy: [0.3375, 0.10875, 0.4, 0.10375, 0.07875, 0.09125, 0.09125, 0.1375, 0.10875, 0.08125]
Start of Epoch 1
Average Train Loss: [2.2348, 1.9242, 2.2247, 1.9196, 1.9252, 1.9186, 1.9239, 2.2088, 1.9218, 1.9162]
Test Accuracy: [0.41625, 0.12625, 0.41625, 0.09375, 0.1025, 0.09375, 0.09375, 0.0975, 0.1075, 0.11125]
Start of Epoch 2
Average Train L

In [29]:
fashion_test_accs_unbalanced = []
fashion_training_times_unbalanced = []
for i in range(2):
    m = MultipleDecoder(10, separate_encoders=True, dataset="fashionmnist", unbalanced=True)
    m.train(dp=False, epochs=20)
    fashion_test_accs_unbalanced.extend([x[-1] for x in m.test_accs])
    fashion_training_times_unbalanced.append(m.training_time)

print(f"Average training time", np.mean(fashion_training_times_unbalanced))
print(f"Average test accuracy", np.mean(fashion_test_accs_unbalanced))
print(f"Median test accuracy", np.median(fashion_test_accs_unbalanced))
print(f"Standard deviation test accuracy", np.std(fashion_test_accs_unbalanced))

[660  60  60  60  60  60  60  60  60  60]
[ 60 660  60  60  60  60  60  60  60  60]
[ 60  60 660  60  60  60  60  60  60  60]
[ 60  60  60 660  60  60  60  60  60  60]
[ 60  60  60  60 660  60  60  60  60  60]
[ 60  60  60  60  60 660  60  60  60  60]
[ 60  60  60  60  60  60 660  60  60  60]
[ 60  60  60  60  60  60  60 660  60  60]
[ 60  60  60  60  60  60  60  60 660  60]
[ 60  60  60  60  60  60  60  60  60 660]
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
Start of Epoch 0
Average Train Loss: [1.9314, 1.9276, 1.9311, 1.9384, 1.9365, 1.9297, 1.941, 1.9423, 1.9247, 1.8861]
Test Accuracy: [0.095, 0.10625, 0.09125, 0.1, 0.1, 0.1, 0.105, 0.115, 0.08875, 0.16625]
Start of Epoch 1
Average Train Loss: [1.9213, 1.9194, 1.9211, 1.9248, 1.9238, 1.9204, 1.9261, 1.9267, 1.9179, 1.8694]
Test Accuracy: [0.11625, 0.10625, 0.07625, 0.0975, 0.09, 0.09875, 0.09875, 0.09, 0.09125, 0.3325]
Start of Epoch 2
Average Train Loss: [1.9179, 1.9

In [30]:
test_accs_dp_unbalanced = []
training_times_dp_unbalanced = []
for i in range(2):
    m = MultipleDecoder(10, separate_encoders=False, dataset="mnist", unbalanced=True)
    m.train(dp=True, epochs=15)
    m.fine_tune(epochs=5)
    test_accs_dp_unbalanced.extend([x[-1] for x in m.test_accs])
    training_times_dp_unbalanced.append(m.training_time)

print(f"Average training time", np.mean(training_times_dp_unbalanced))
print(f"Average test accuracy", np.mean(test_accs_dp_unbalanced))
print(f"Median test accuracy", np.median(test_accs_dp_unbalanced))
print(f"Standard deviation test accuracy", np.std(test_accs_dp_unbalanced))

[660  60  60  60  60  60  60  60  60  60]
[ 60 660  60  60  60  60  60  60  60  60]
[ 60  60 660  60  60  60  60  60  60  60]
[ 60  60  60 660  60  60  60  60  60  60]
[ 60  60  60  60 660  60  60  60  60  60]
[ 60  60  60  60  60 660  60  60  60  60]
[ 60  60  60  60  60  60 660  60  60  60]
[ 60  60  60  60  60  60  60 660  60  60]
[ 60  60  60  60  60  60  60  60 660  60]
[ 60  60  60  60  60  60  60  60  60 660]
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
Start of Epoch 0
Average Train Loss: [1.9258, 1.9297, 1.9327, 2.3036, 2.2493, 1.928, 1.9346, 1.9186, 1.925, 1.9307]
Test Accuracy: [0.09625, 0.1075, 0.10875, 0.08125, 0.3275, 0.1, 0.09375, 0.11375, 0.09875, 0.09625]
Start of Epoch 1
Average Train Loss: [1.9185, 1.9204, 1.9219, 2.3031, 2.2268, 1.9196, 1.9229, 1.9149, 1.9181, 1.9209]
Test Accuracy: [0.0925, 0.105, 0.09, 0.12, 0.27625, 0.0875, 0.08875, 0.1075, 0.09625, 0.1]
Start of Epoch 2
Average Train Loss: [1.916, 

In [31]:
fashion_test_accs_dp_unbalanced = []
fashion_training_times_dp_unbalanced = []
for i in range(2):
    m = MultipleDecoder(10, separate_encoders=False, dataset="fashionmnist", unbalanced=True)
    m.train(dp=True, epochs=15)
    m.fine_tune(epochs=5)
    fashion_test_accs_dp_unbalanced.extend([x[-1] for x in m.test_accs])
    fashion_training_times_dp_unbalanced.append(m.training_time)

print(f"Average training time", np.mean(fashion_training_times_dp_unbalanced))
print(f"Average test accuracy", np.mean(fashion_test_accs_dp_unbalanced))
print(f"Median test accuracy", np.median(fashion_test_accs_dp_unbalanced))
print(f"Standard deviation test accuracy", np.std(fashion_test_accs_dp_unbalanced))

[660  60  60  60  60  60  60  60  60  60]
[ 60 660  60  60  60  60  60  60  60  60]
[ 60  60 660  60  60  60  60  60  60  60]
[ 60  60  60 660  60  60  60  60  60  60]
[ 60  60  60  60 660  60  60  60  60  60]
[ 60  60  60  60  60 660  60  60  60  60]
[ 60  60  60  60  60  60 660  60  60  60]
[ 60  60  60  60  60  60  60 660  60  60]
[ 60  60  60  60  60  60  60  60 660  60]
[ 60  60  60  60  60  60  60  60  60 660]
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
1200
10000
Start of Epoch 0
Average Train Loss: [1.9345, 1.9265, 1.9275, 1.9317, 1.9306, 1.9757, 1.9193, 1.9264, 1.921, 1.9052]
Test Accuracy: [0.1075, 0.0875, 0.11625, 0.08875, 0.105, 0.0975, 0.0975, 0.09875, 0.10875, 0.2775]
Start of Epoch 1
Average Train Loss: [1.9228, 1.9188, 1.9193, 1.9214, 1.9209, 1.9434, 1.9152, 1.9188, 1.9161, 1.8688]
Test Accuracy: [0.08375, 0.0825, 0.095, 0.09875, 0.08875, 0.1025, 0.12375, 0.09, 0.11625, 0.18375]
Start of Epoch 2
Average Train Loss: 

In [41]:
test_accs_dp50 = []
training_times_dp50 = []
for i in range(1):
    m = MultipleDecoder(50, separate_encoders=False, separate_decoders=False)
    m.train(dp=True, epochs=20)
    #m.fine_tune(epochs=5)
    test_accs_dp50.extend([x[-1] for x in m.test_accs])
    training_times_dp50.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp50))
print(f"Average test accuracy", np.mean(test_accs_dp50))
print(f"Median test accuracy", np.median(test_accs_dp50))
print(f"Standard deviation test accuracy", np.std(test_accs_dp50))

data slice [5741, ..., 39095]
data slice [10556, ..., 10595]
data slice [24713, ..., 20261]
data slice [284, ..., 51443]
data slice [9303, ..., 25893]
data slice [11041, ..., 8656]
data slice [19335, ..., 40281]
data slice [5724, ..., 17180]
data slice [1463, ..., 58063]
data slice [57553, ..., 37155]
data slice [2977, ..., 29181]
data slice [2082, ..., 15769]
data slice [20986, ..., 6257]
data slice [36067, ..., 7924]
data slice [51302, ..., 10306]
data slice [29597, ..., 40087]
data slice [33555, ..., 6780]
data slice [45772, ..., 57534]
data slice [3066, ..., 8092]
data slice [53606, ..., 29401]
data slice [2862, ..., 37293]
data slice [8480, ..., 13963]
data slice [37957, ..., 6341]
data slice [17912, ..., 33918]
data slice [26143, ..., 39162]
data slice [13637, ..., 58716]
data slice [46469, ..., 35030]
data slice [57241, ..., 29512]
data slice [19921, ..., 12522]
data slice [16303, ..., 2120]
data slice [18242, ..., 22252]
data slice [3121, ..., 42728]
data slice [55566, ..., 171

KeyboardInterrupt: 

In [ ]:
test_accs_dp50 = []
training_times_dp50 = []
for i in range(1):
    m = MultipleDecoder(50, separate_encoders=False, separate_decoders=True)
    m.train(dp=True, epochs=15)
    m.fine_tune(epochs=5)
    test_accs_dp50.extend([x[-1] for x in m.test_accs])
    training_times_dp50.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp50))
print(f"Average test accuracy", np.mean(test_accs_dp50))
print(f"Median test accuracy", np.median(test_accs_dp50))
print(f"Standard deviation test accuracy", np.std(test_accs_dp50))

In [55]:
test_accs_dp50_model2 = []
training_times_dp50_model2 = []
for i in range(1):
    m = MultipleDecoder(50, separate_encoders=False, separate_decoders=True, first_models=False)
    m.train(dp=True, epochs=10)
    #m.fine_tune(epochs=5)
    test_accs_dp50_model2.extend([x[-1] for x in m.test_accs])
    training_times_dp50_model2.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp50_model2))
print(f"Average test accuracy", np.mean(test_accs_dp50_model2))
print(f"Median test accuracy", np.median(test_accs_dp50_model2))
print(f"Standard deviation test accuracy", np.std(test_accs_dp50_model2))

Second Models
<class '__main__.EncoderCNN2'>
<class '__main__.DecoderCNN2'>
data slice [59777, ..., 37279]
data slice [41248, ..., 45637]
data slice [15168, ..., 54969]
data slice [10195, ..., 21091]
data slice [41456, ..., 14622]
data slice [37525, ..., 25592]
data slice [12347, ..., 45574]
data slice [8652, ..., 18556]
data slice [34738, ..., 17517]
data slice [10270, ..., 3954]
data slice [19384, ..., 50780]
data slice [52044, ..., 12710]
data slice [50291, ..., 51349]
data slice [33569, ..., 71]
data slice [24889, ..., 9777]
data slice [37978, ..., 8065]
data slice [13147, ..., 494]
data slice [8181, ..., 13964]
data slice [37635, ..., 10193]
data slice [36098, ..., 44051]
data slice [34847, ..., 42279]
data slice [19829, ..., 30986]
data slice [46516, ..., 497]
data slice [13728, ..., 35331]
data slice [58696, ..., 51614]
data slice [11451, ..., 14766]
data slice [49805, ..., 58196]
data slice [34117, ..., 41212]
data slice [26638, ..., 28394]
data slice [48011, ..., 23695]
data s

In [58]:
test_accs_dp50 = []
training_times_dp50 = []
for i in range(1):
    m = MultipleDecoder(50, separate_encoders=False, separate_decoders=False, first_models=False)
    m.train(dp=False, epochs=5)
    #m.fine_tune(epochs=5)
    test_accs_dp50.extend([x[-1] for x in m.test_accs])
    training_times_dp50.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp50))
print(f"Average test accuracy", np.mean(test_accs_dp50))
print(f"Median test accuracy", np.median(test_accs_dp50))
print(f"Standard deviation test accuracy", np.std(test_accs_dp50))

Second Models
<class '__main__.EncoderCNN2'>
<class '__main__.DecoderCNN2'>
data slice [47387, ..., 50411]
data slice [38402, ..., 11526]
data slice [34259, ..., 31960]
data slice [55124, ..., 46603]
data slice [14784, ..., 7840]
data slice [19025, ..., 34650]
data slice [40248, ..., 4900]
data slice [27165, ..., 6902]
data slice [31486, ..., 23430]
data slice [22183, ..., 56559]
data slice [13332, ..., 56538]
data slice [16903, ..., 12997]
data slice [43440, ..., 19179]
data slice [6403, ..., 6641]
data slice [47579, ..., 24518]
data slice [40815, ..., 53825]
data slice [59186, ..., 10152]
data slice [17392, ..., 39594]
data slice [49178, ..., 42996]
data slice [7488, ..., 16110]
data slice [30477, ..., 18050]
data slice [17576, ..., 33427]
data slice [42399, ..., 57708]
data slice [17659, ..., 17510]
data slice [45679, ..., 31855]
data slice [9445, ..., 44348]
data slice [39660, ..., 1626]
data slice [15229, ..., 30402]
data slice [49767, ..., 17347]
data slice [16723, ..., 32576]
da

In [61]:
test_accs_dp50 = []
training_times_dp50 = []
for i in range(1):
    m = MultipleDecoder(50, separate_encoders=False, separate_decoders=False, first_models=False)
    m.train(dp=False, epochs=5)
    #m.fine_tune(epochs=5)
    test_accs_dp50.extend([x[-1] for x in m.test_accs])
    training_times_dp50.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp50))
print(f"Average test accuracy", np.mean(test_accs_dp50))
print(f"Median test accuracy", np.median(test_accs_dp50))
print(f"Standard deviation test accuracy", np.std(test_accs_dp50))

Second Models
<class '__main__.EncoderCNN2'>
<class '__main__.DecoderCNN2'>
data slice [48951, ..., 51895]
data slice [288, ..., 28838]
data slice [44617, ..., 21906]
data slice [48743, ..., 7754]
data slice [19662, ..., 27489]
data slice [45624, ..., 49295]
data slice [4051, ..., 14852]
data slice [39420, ..., 37271]
data slice [39689, ..., 47073]
data slice [20887, ..., 7802]
data slice [42985, ..., 48375]
data slice [5392, ..., 34999]
data slice [9972, ..., 36040]
data slice [2869, ..., 6763]
data slice [28842, ..., 32444]
data slice [35699, ..., 27398]
data slice [21472, ..., 38715]
data slice [53107, ..., 11227]
data slice [22461, ..., 57300]
data slice [5065, ..., 52706]
data slice [6894, ..., 52505]
data slice [26498, ..., 16571]
data slice [36814, ..., 21050]
data slice [51510, ..., 54366]
data slice [8054, ..., 25817]
data slice [18960, ..., 1233]
data slice [7190, ..., 16605]
data slice [50142, ..., 19033]
data slice [51322, ..., 27584]
data slice [37068, ..., 7364]
data slic

KeyboardInterrupt: 

In [80]:
test_accs_dp50 = []
training_times_dp50 = []
for i in range(1):
    m = MultipleDecoder(50, separate_encoders=False, separate_decoders=False, first_models=False)
    m.train(dp=True, epochs=5)
    #m.fine_tune(epochs=5)
    test_accs_dp50.extend([x[-1] for x in m.test_accs])
    training_times_dp50.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp50))
print(f"Average test accuracy", np.mean(test_accs_dp50))
print(f"Median test accuracy", np.median(test_accs_dp50))
print(f"Standard deviation test accuracy", np.std(test_accs_dp50))

Second Models
<class '__main__.EncoderCNN2'>
<class '__main__.DecoderCNN2'>
data slice [33286, ..., 41292]
data slice [34415, ..., 27939]
data slice [25912, ..., 45452]
data slice [59661, ..., 55808]
data slice [38928, ..., 23294]
data slice [53778, ..., 4832]
data slice [34234, ..., 30148]
data slice [3416, ..., 10645]
data slice [44282, ..., 35642]
data slice [25464, ..., 11777]
data slice [18876, ..., 4946]
data slice [47729, ..., 15338]
data slice [11697, ..., 17919]
data slice [41584, ..., 41483]
data slice [25309, ..., 35004]
data slice [7838, ..., 42039]
data slice [47036, ..., 38068]
data slice [23779, ..., 55128]
data slice [8551, ..., 5642]
data slice [26662, ..., 32384]
data slice [17249, ..., 16789]
data slice [47554, ..., 52650]
data slice [25768, ..., 37640]
data slice [14239, ..., 29599]
data slice [45384, ..., 45976]
data slice [47772, ..., 6640]
data slice [58639, ..., 38242]
data slice [46334, ..., 4548]
data slice [24565, ..., 29765]
data slice [15701, ..., 43398]
da

In [81]:
test_accs_dp50 = []
training_times_dp50 = []
for i in range(1):
    m = MultipleDecoder(50, separate_encoders=False, separate_decoders=False, first_models=False, epsilon=1e5)
    m.train(dp=True, epochs=5)
    #m.fine_tune(epochs=5)
    test_accs_dp50.extend([x[-1] for x in m.test_accs])
    training_times_dp50.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp50))
print(f"Average test accuracy", np.mean(test_accs_dp50))
print(f"Median test accuracy", np.median(test_accs_dp50))
print(f"Standard deviation test accuracy", np.std(test_accs_dp50))

Second Models
<class '__main__.EncoderCNN2'>
<class '__main__.DecoderCNN2'>
data slice [45148, ..., 46796]
data slice [28511, ..., 141]
data slice [49368, ..., 58275]
data slice [39335, ..., 34017]
data slice [39537, ..., 16037]
data slice [58780, ..., 31033]
data slice [8599, ..., 28540]
data slice [25260, ..., 921]
data slice [33580, ..., 24848]
data slice [3023, ..., 44457]
data slice [6650, ..., 24919]
data slice [30697, ..., 26891]
data slice [50977, ..., 1882]
data slice [45833, ..., 59446]
data slice [24520, ..., 52656]
data slice [25581, ..., 7045]
data slice [33943, ..., 30192]
data slice [7477, ..., 8074]
data slice [39441, ..., 41724]
data slice [1050, ..., 18636]
data slice [2272, ..., 4349]
data slice [16344, ..., 57775]
data slice [26781, ..., 11060]
data slice [20857, ..., 6307]
data slice [34107, ..., 58433]
data slice [38652, ..., 49018]
data slice [2003, ..., 34918]
data slice [14847, ..., 21043]
data slice [10310, ..., 59596]
data slice [25375, ..., 57962]
data slice

In [82]:
test_accs_dp50 = []
training_times_dp50 = []
for i in range(1):
    m = MultipleDecoder(50, separate_encoders=False, separate_decoders=False, first_models=False, epsilon=1e4)
    m.train(dp=True, epochs=5)
    #m.fine_tune(epochs=5)
    test_accs_dp50.extend([x[-1] for x in m.test_accs])
    training_times_dp50.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp50))
print(f"Average test accuracy", np.mean(test_accs_dp50))
print(f"Median test accuracy", np.median(test_accs_dp50))
print(f"Standard deviation test accuracy", np.std(test_accs_dp50))

Second Models
<class '__main__.EncoderCNN2'>
<class '__main__.DecoderCNN2'>
data slice [15066, ..., 43241]
data slice [13611, ..., 3283]
data slice [58943, ..., 25935]
data slice [46744, ..., 40816]
data slice [59200, ..., 17881]
data slice [12701, ..., 49888]
data slice [39311, ..., 2300]
data slice [31170, ..., 46435]
data slice [8387, ..., 38056]
data slice [12254, ..., 11408]
data slice [58422, ..., 35041]
data slice [1934, ..., 699]
data slice [24116, ..., 150]
data slice [43974, ..., 16966]
data slice [53943, ..., 51544]
data slice [22945, ..., 40589]
data slice [35001, ..., 17535]
data slice [8283, ..., 29172]
data slice [28075, ..., 20472]
data slice [23843, ..., 44753]
data slice [32746, ..., 17587]
data slice [52109, ..., 9974]
data slice [47416, ..., 22551]
data slice [28557, ..., 55893]
data slice [52687, ..., 46598]
data slice [48308, ..., 11771]
data slice [10585, ..., 23942]
data slice [2503, ..., 26624]
data slice [59400, ..., 4835]
data slice [42481, ..., 58177]
data s

In [83]:
test_accs_dp50 = []
training_times_dp50 = []
for i in range(1):
    m = MultipleDecoder(50, separate_encoders=False, separate_decoders=False, first_models=False, epsilon=1e3)
    m.train(dp=True, epochs=5)
    #m.fine_tune(epochs=5)
    test_accs_dp50.extend([x[-1] for x in m.test_accs])
    training_times_dp50.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp50))
print(f"Average test accuracy", np.mean(test_accs_dp50))
print(f"Median test accuracy", np.median(test_accs_dp50))
print(f"Standard deviation test accuracy", np.std(test_accs_dp50))

Second Models
<class '__main__.EncoderCNN2'>
<class '__main__.DecoderCNN2'>
data slice [51995, ..., 33509]
data slice [16268, ..., 14633]
data slice [57370, ..., 18119]
data slice [40933, ..., 25656]
data slice [14007, ..., 4418]
data slice [14904, ..., 37527]
data slice [23811, ..., 6122]
data slice [50594, ..., 10511]
data slice [43394, ..., 25894]
data slice [20789, ..., 44529]
data slice [18194, ..., 11372]
data slice [11898, ..., 13684]
data slice [24080, ..., 33605]
data slice [6606, ..., 59509]
data slice [52596, ..., 46896]
data slice [17649, ..., 8488]
data slice [35289, ..., 16487]
data slice [47329, ..., 31557]
data slice [13767, ..., 4245]
data slice [41336, ..., 6132]
data slice [58886, ..., 3983]
data slice [52201, ..., 40382]
data slice [41403, ..., 37320]
data slice [29735, ..., 7076]
data slice [43277, ..., 54617]
data slice [49094, ..., 58138]
data slice [29605, ..., 30846]
data slice [4054, ..., 33357]
data slice [2792, ..., 44610]
data slice [50058, ..., 54026]
data

In [105]:
class EmptyDecoder:
    def __init__(self, n, hidden_size=28, device=DEVICE, alpha=20, epsilon=1e6, first_models=True, separate_encoders=False, separate_decoders=False, dataset="mnist", same_decoder_init=False, unbalanced=False):
        """
        Creates n decoders for single encoder
        """
        self.device = device
        self.n = n
        self.hidden_size = hidden_size
        self.alpha = alpha
        self.epsilon = epsilon
        self.dataset = dataset
        
        if not first_models:
            print("Second Models")
            EncoderCNN = EncoderCNN2
            DecoderCNN = DecoderCNN2
            print(EncoderCNN)
            print(DecoderCNN)
        else:
            EncoderCNN = EncoderCNN2
            DecoderCNN = DecoderCNN
        
        self.separate_encoders = bool(separate_encoders)
        self.separate_decoders = bool(separate_decoders)
        if self.separate_encoders:
            self.encoders = []
            self.encoder_optimizers = []
            for i in range(self.n):
                self.encoders.append(EncoderCNN(1, self.hidden_size).to(self.device))
                self.encoder_optimizers.append(optim.Adam(self.encoders[i].parameters(),lr=0.001))
        else:
            self.encoder = EncoderCNN(1, self.hidden_size).to(self.device)
            self.encoder_optimizer = optim.Adam(self.encoder.parameters(),lr=0.001)
        
        self.decoder = DecoderCNN(self.hidden_size).to(self.device)
        self.decoder_criterion = nn.CrossEntropyLoss()
        
        if unbalanced:
            self.train_data_generators, self.test_data_generators = multiple_data_generators_unbalanced(self.n, batch_sizes=16, dataset=dataset)
        else:
            self.train_data_generators, self.test_data_generators = multiple_data_generators(self.n, batch_sizes=16, dataset=dataset)

    def train(self, epochs=20, train_steps_per_epoch=75, test_steps_per_epoch=50, final_test_steps=625, randomize_order=False, dp=True):
        self._train_init(epochs=epochs, train_steps_per_epoch=train_steps_per_epoch, test_steps_per_epoch=test_steps_per_epoch, randomize_order=randomize_order)
        
        import time
        start = time.time()
        elapsed_test_time = 0.0
        while True:
            if self.model_steps >= (self.train_steps_per_epoch*self.epochs):
                print("Finished Training")
                break
            if self.model_steps % self.train_steps_per_epoch == 0:
                print("Start of Epoch", (self.model_steps // self.train_steps_per_epoch))
  
            self._train_batch(dp=dp)

            if self.model_steps % self.train_steps_per_epoch == 0:
                train_losses = [round(10000 * np.mean(x))/10000 for x in self.train_losses]
                print(f"Average Train Loss:", train_losses)
                #for i in range(self.n):
                #    print(f"Average Train Loss for {i}:", np.mean(self.train_losses[i]))
                
                if test_steps_per_epoch:
                    start_test = time.time()
                    for i in range(self.n):
                        if self.separate_encoders:
                            encoder = self.encoders[i]
                        else:
                            encoder = self.encoder
                        
                        decoder = self.decoder
                        self.test_accs[i].append(encdec_accuracy(encoder, decoder, self.test_data_generators[i], self.test_steps_per_epoch, device=self.device))
                
                    test_accs = [x[-1] for x in self.test_accs]
                    print(f"Test Accuracy:", test_accs)
                    end_test = time.time()
                    elapsed_test_time += end_test - start_test
                #for i in range(self.n):
                #    print(f"Test Accuracy {i}:", self.test_accs[i][-1])
        
        end = time.time()
        elapsed = end - start
        self.training_time = elapsed - elapsed_test_time
        print(f"Elapsed Test Time:", elapsed_test_time)
        print(f"Elapsed Training Time:", self.training_time)
        for i in range(self.n):
            if self.separate_encoders:
                encoder = self.encoders[i]
            else:
                encoder = self.encoder
            decoder = self.decoder
            self.test_accs[i].append(encdec_accuracy(encoder, decoder, self.test_data_generators[i], final_test_steps, device=self.device))
            
        test_accs = [x[-1] for x in self.test_accs]
        print(f"Final Test Accuracy:", test_accs)
    
    def _train_init(self, epochs=20, train_steps_per_epoch=375, test_steps_per_epoch=100, randomize_order=False):
        self.epochs = epochs
        self.model_steps = 0
        if self.alpha is None or self.epsilon is None:
            self.epsilon_iter = None
        else:
            self.epsilon_iter = self.epsilon / self.epochs
        
        self.train_steps_per_epoch = train_steps_per_epoch
        self.test_steps_per_epoch = test_steps_per_epoch
        self.randomize_order = randomize_order
        
        # plotting criteria
        self.train_losses = [[] for x in range(self.n)]
        self.test_accs = [[] for x in range(self.n)]        

    def _train_batch(self, dp=True):
        index = list(range(self.n))
        if self.randomize_order:
            np.random.shuffle(index)
        
        for i in index:
            if dp:
                self._train_decoder_batch_dp(i)
            else:
                self._train_decoder_batch_no_dp(i)
        
        self.model_steps += 1
        
    def _train_decoder_batch_no_dp(self, i):
        """
        Train decoder i on the next batch without differential privacy
        """
        if self.separate_encoders:
            encoder = self.encoders[i]
            encoder_optimizer = self.encoder_optimizers[i]
        else:
            encoder = self.encoder
            encoder_optimizer = self.encoder_optimizer
        
        decoder = self.decoder
        decoder_criterion = self.decoder_criterion
        
        x_batch_train, y_batch_train = (x.to(self.device) for x in next(self.train_data_generators[i]))
        encoder_optimizer.zero_grad()
        
        # compute loss
        embedding = encoder.forward(x_batch_train)
        outputs = decoder.forward(embedding)
        loss = decoder_criterion(outputs, y_batch_train)

        loss.backward()
        self.train_losses[i].append(loss.item())

        # perform the a gradient step
        encoder_optimizer.step()
        
    def _train_decoder_batch_dp(self, i):
        """
        Train decoder i on the next batch using differential privacy
        """
        if self.separate_encoders:
            encoder = self.encoders[i]
            encoder_optimizer = self.encoder_optimizers[i]
        else:
            encoder = self.encoder
            encoder_optimizer = self.encoder_optimizer
        
        decoder = self.decoder
        decoder_criterion = self.decoder_criterion
        
        x_batch_train, y_batch_train = (x.to(self.device) for x in next(self.train_data_generators[i]))
        encoder_optimizer.zero_grad()
        
        # Need grad on input for sensitivity; not sure if cloning is needed
        x_batch_train = torch.autograd.Variable(torch.clone(x_batch_train).to(self.device), requires_grad=True)
        
        # compute loss
        embedding = encoder.forward(x_batch_train)
        outputs = decoder.forward(embedding)
        loss = decoder_criterion(outputs, y_batch_train)
        
        batch_sensitivities = compute_immediate_sensitivity(
            encoder, x_batch_train, loss
        )
        batch_sensitivity = torch.max(batch_sensitivities) / len(x_batch_train)
        
        loss.backward()
        # step 4. compute noise
        # this is the scale of the Gaussian noise to be added to the batch gradient
        sigma = torch.sqrt(
            (batch_sensitivity ** 2 * self.alpha) / (2 * self.epsilon_iter)
        )
        
        # logging
        #x = loss.item()
        #if np.isnan(x):
        #    log.debug(
        #        f"Epoch (before adj): {epoch}; batch_idx: {batch_idx} loss: {loss}"
        #    )
        #    x = 0
        #    log.debug(
        #        f"    epoch (adfter adj): {epoch}; batch_idx: {batch_idx} loss: {loss}"
        #    )
        #train_loss += x
        
        self.train_losses[i].append(loss.item())
        
        # step 5. update gradients with computed sensitivities
        with torch.no_grad():
            for p in encoder.parameters():
                p.grad += sigma * torch.randn(1).to(self.device)
        
        # perform the a gradient step
        encoder_optimizer.step()

    def fine_tune(self, dp=False, **kwargs):
        """
        Used to fine tune individual encoder / decoder pairs AFTER primary collective training
        """
        import copy
        self.separate_encoders = True
        if self.separate_encoders:
            self.encoders = []
            self.encoder_optimizers = []
            for i in range(self.n):
                self.encoders.append(copy.deepcopy(self.encoder).to(self.device))
                self.encoder_optimizers.append(optim.Adam(self.encoders[i].parameters(),lr=0.001))

        self.training_time_dp = self.training_time
        self.test_accs_dp = self.test_accs
        self.train(dp=dp, **kwargs)
        
        self.training_time += self.training_time_dp

In [108]:
test_accs_dp10 = []
training_times_dp10 = []
for i in range(1):
    m = EmptyDecoder(10, separate_encoders=False, separate_decoders=False, first_models=False, epsilon=1e6)
    m.train(dp=True, epochs=10)
    #m.fine_tune(epochs=5)
    test_accs_dp10.extend([x[-1] for x in m.test_accs])
    training_times_dp10.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp10))
print(f"Average test accuracy", np.mean(test_accs_dp10))
print(f"Median test accuracy", np.median(test_accs_dp10))
print(f"Standard deviation test accuracy", np.std(test_accs_dp10))

Second Models
<class '__main__.EncoderCNN2'>
<class '__main__.DecoderCNN2'>
data slice [46832, ..., 1555]
data slice [31874, ..., 10270]
data slice [47191, ..., 37569]
data slice [56936, ..., 48741]
data slice [20658, ..., 58834]
data slice [22692, ..., 42243]
data slice [59251, ..., 39880]
data slice [45659, ..., 7007]
data slice [19656, ..., 14762]
data slice [49080, ..., 59971]
Start of Epoch 0
Average Train Loss: [1.8211, 1.8385, 1.8106, 1.817, 1.8412, 1.8141, 1.8226, 1.8204, 1.8336, 1.8349]
Test Accuracy: [0.765, 0.78875, 0.7925, 0.7725, 0.7925, 0.7875, 0.76375, 0.79, 0.80125, 0.79]
Start of Epoch 1
Average Train Loss: [1.7218, 1.7361, 1.7088, 1.7136, 1.7385, 1.7165, 1.7221, 1.7167, 1.7317, 1.7276]
Test Accuracy: [0.8775, 0.85125, 0.87, 0.85375, 0.83875, 0.8525, 0.8425, 0.85125, 0.865, 0.8625]
Start of Epoch 2
Average Train Loss: [1.6816, 1.6905, 1.6694, 1.6742, 1.6946, 1.6765, 1.6812, 1.6777, 1.6904, 1.6852]
Test Accuracy: [0.85, 0.84625, 0.8525, 0.8625, 0.855, 0.85375, 0.85125, 

In [99]:
test_accs_dp10 = []
training_times_dp10 = []
for i in range(1):
    m = EmptyDecoder(10, separate_encoders=False, separate_decoders=False, first_models=False, epsilon=2e5)
    m.train(dp=True, epochs=10)
    #m.fine_tune(epochs=5)
    test_accs_dp10.extend([x[-1] for x in m.test_accs])
    training_times_dp10.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp10))
print(f"Average test accuracy", np.mean(test_accs_dp10))
print(f"Median test accuracy", np.median(test_accs_dp10))
print(f"Standard deviation test accuracy", np.std(test_accs_dp10))

Second Models
<class '__main__.EncoderCNN2'>
<class '__main__.DecoderCNN2'>
data slice [48846, ..., 13431]
data slice [7406, ..., 39944]
data slice [25688, ..., 34640]
data slice [10729, ..., 6981]
data slice [29883, ..., 14167]
data slice [59553, ..., 1290]
data slice [58437, ..., 38177]
data slice [31837, ..., 2395]
data slice [11147, ..., 51147]
data slice [44380, ..., 24510]
Start of Epoch 0
Average Train Loss: [1.8089, 1.7899, 1.8149, 1.7743, 1.7849, 1.7919, 1.7658, 1.762, 1.7857, 1.7878]
Test Accuracy: [0.76125, 0.795, 0.7575, 0.75625, 0.77875, 0.76125, 0.7525, 0.77375, 0.74375, 0.755]
Start of Epoch 1
Average Train Loss: [1.7475, 1.7349, 1.7506, 1.7119, 1.729, 1.7346, 1.71, 1.7173, 1.7325, 1.7258]
Test Accuracy: [0.8325, 0.8325, 0.81875, 0.84875, 0.84375, 0.83375, 0.83125, 0.83875, 0.84125, 0.86125]
Start of Epoch 2
Average Train Loss: [1.7097, 1.7015, 1.7133, 1.6802, 1.6972, 1.7009, 1.6754, 1.6813, 1.6969, 1.6912]
Test Accuracy: [0.82125, 0.84375, 0.83125, 0.825, 0.83625, 0.838

In [106]:
test_accs_dp10 = []
training_times_dp10 = []
for i in range(1):
    m = EmptyDecoder(10, separate_encoders=False, separate_decoders=False, first_models=False, epsilon=1e5)
    m.train(dp=True, epochs=10)
    #m.fine_tune(epochs=2)
    test_accs_dp10.extend([x[-1] for x in m.test_accs])
    training_times_dp10.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp10))
print(f"Average test accuracy", np.mean(test_accs_dp10))
print(f"Median test accuracy", np.median(test_accs_dp10))
print(f"Standard deviation test accuracy", np.std(test_accs_dp10))

Second Models
<class '__main__.EncoderCNN2'>
<class '__main__.DecoderCNN2'>
data slice [48304, ..., 6709]
data slice [53129, ..., 48192]
data slice [26819, ..., 40633]
data slice [37345, ..., 21909]
data slice [27175, ..., 7624]
data slice [34167, ..., 358]
data slice [37179, ..., 52088]
data slice [2327, ..., 29179]
data slice [37973, ..., 44675]
data slice [58255, ..., 24982]
Start of Epoch 0
Average Train Loss: [1.7832, 1.7838, 1.7893, 1.778, 1.7763, 1.7809, 1.7678, 1.7741, 1.7652, 1.765]
Test Accuracy: [0.79, 0.78875, 0.78, 0.7675, 0.78625, 0.775, 0.7675, 0.76875, 0.81, 0.78]
Start of Epoch 1
Average Train Loss: [1.7466, 1.7422, 1.7517, 1.7432, 1.7343, 1.7445, 1.7306, 1.7421, 1.7302, 1.7293]
Test Accuracy: [0.715, 0.75375, 0.7425, 0.72875, 0.7425, 0.725, 0.7625, 0.71, 0.75, 0.76125]
Start of Epoch 2
Average Train Loss: [1.7222, 1.7181, 1.7248, 1.7189, 1.7124, 1.7179, 1.7039, 1.7196, 1.7062, 1.7055]
Test Accuracy: [0.82, 0.8175, 0.81875, 0.8075, 0.8275, 0.83, 0.81125, 0.82125, 0.816

In [101]:
test_accs_dp10 = []
training_times_dp10 = []
for i in range(1):
    m = EmptyDecoder(10, separate_encoders=False, separate_decoders=False, first_models=False, epsilon=1e4)
    m.train(dp=True, epochs=10)
    #m.fine_tune(epochs=5)
    test_accs_dp10.extend([x[-1] for x in m.test_accs])
    training_times_dp10.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp10))
print(f"Average test accuracy", np.mean(test_accs_dp10))
print(f"Median test accuracy", np.median(test_accs_dp10))
print(f"Standard deviation test accuracy", np.std(test_accs_dp10))

Second Models
<class '__main__.EncoderCNN2'>
<class '__main__.DecoderCNN2'>
data slice [49153, ..., 48891]
data slice [22724, ..., 57320]
data slice [42540, ..., 16539]
data slice [31826, ..., 34647]
data slice [18616, ..., 24885]
data slice [44709, ..., 4970]
data slice [38983, ..., 34016]
data slice [25328, ..., 42468]
data slice [17899, ..., 57963]
data slice [19216, ..., 2970]
Start of Epoch 0
Average Train Loss: [1.9269, 1.9095, 1.9013, 1.9404, 1.8973, 1.9184, 1.9038, 1.9147, 1.8994, 1.8902]
Test Accuracy: [0.67125, 0.6725, 0.66375, 0.64125, 0.6675, 0.64375, 0.65, 0.66375, 0.6725, 0.64125]
Start of Epoch 1
Average Train Loss: [1.8777, 1.857, 1.8483, 1.8882, 1.8557, 1.8703, 1.8612, 1.8648, 1.8588, 1.8416]
Test Accuracy: [0.6825, 0.67375, 0.67125, 0.65875, 0.665, 0.6675, 0.66625, 0.65375, 0.65625, 0.6725]
Start of Epoch 2
Average Train Loss: [1.8586, 1.8392, 1.83, 1.8678, 1.8406, 1.85, 1.845, 1.8477, 1.841, 1.8251]
Test Accuracy: [0.6275, 0.6575, 0.69, 0.69375, 0.6525, 0.7, 0.66, 0.

In [109]:
test_accs_dp10 = []
training_times_dp10 = []
for i in range(1):
    m = EmptyDecoder(10, separate_encoders=False, separate_decoders=False, first_models=False, epsilon=1e3)
    m.train(dp=True, epochs=10)
    #m.fine_tune(epochs=5)
    test_accs_dp10.extend([x[-1] for x in m.test_accs])
    training_times_dp10.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp10))
print(f"Average test accuracy", np.mean(test_accs_dp10))
print(f"Median test accuracy", np.median(test_accs_dp10))
print(f"Standard deviation test accuracy", np.std(test_accs_dp10))

Second Models
<class '__main__.EncoderCNN2'>
<class '__main__.DecoderCNN2'>
data slice [8858, ..., 51403]
data slice [38373, ..., 58069]
data slice [41965, ..., 59917]
data slice [11494, ..., 25615]
data slice [26869, ..., 25932]
data slice [9763, ..., 59144]
data slice [51701, ..., 8538]
data slice [59592, ..., 2909]
data slice [17898, ..., 42958]
data slice [34255, ..., 14848]
Start of Epoch 0
Average Train Loss: [2.1687, 2.1735, 2.1769, 2.1635, 2.1695, 2.1755, 2.1676, 2.1559, 2.1614, 2.1792]
Test Accuracy: [0.37125, 0.35375, 0.35625, 0.35, 0.3575, 0.3425, 0.34875, 0.3875, 0.3675, 0.36]
Start of Epoch 1
Average Train Loss: [2.1353, 2.1384, 2.1368, 2.1243, 2.1474, 2.1424, 2.1254, 2.1226, 2.1279, 2.1523]
Test Accuracy: [0.355, 0.3525, 0.3475, 0.395, 0.3675, 0.37625, 0.37375, 0.3575, 0.33875, 0.37875]
Start of Epoch 2
Average Train Loss: [2.0787, 2.0785, 2.0818, 2.0699, 2.0956, 2.0868, 2.0716, 2.0746, 2.0747, 2.1006]
Test Accuracy: [0.6225, 0.65125, 0.64, 0.65125, 0.63, 0.61, 0.62, 0.62

In [161]:
test_accs_dp10 = []
training_times_dp10 = []
for i in range(1):
    m = EmptyDecoder(10, separate_encoders=False, separate_decoders=False, first_models=False, epsilon=1e2)
    m.train(dp=True, epochs=10)
    #m.fine_tune(epochs=5)
    test_accs_dp10.extend([x[-1] for x in m.test_accs])
    training_times_dp10.append(m.training_time)
print(f"Average training time", np.mean(training_times_dp10))
print(f"Average test accuracy", np.mean(test_accs_dp10))
print(f"Median test accuracy", np.median(test_accs_dp10))
print(f"Standard deviation test accuracy", np.std(test_accs_dp10))

Second Models
<class '__main__.EncoderCNN2'>
<class '__main__.DecoderCNN2'>
data slice [45990, ..., 30972]
data slice [43637, ..., 39553]
data slice [50585, ..., 29423]
data slice [43735, ..., 53254]
data slice [34567, ..., 30669]
data slice [33318, ..., 8847]
data slice [37541, ..., 11666]
data slice [5517, ..., 42705]
data slice [43474, ..., 20326]
data slice [58052, ..., 34057]
Start of Epoch 0
Average Train Loss: [2.0628, 2.0485, 2.0595, 2.0697, 2.0808, 2.0722, 2.0482, 2.0656, 2.0789, 2.0342]
Test Accuracy: [0.4875, 0.49625, 0.48625, 0.4725, 0.45375, 0.48375, 0.4825, 0.45375, 0.48875, 0.50875]
Start of Epoch 1
Average Train Loss: [2.0816, 2.0753, 2.0729, 2.0873, 2.097, 2.0878, 2.0664, 2.0855, 2.0967, 2.061]
Test Accuracy: [0.37125, 0.35375, 0.385, 0.39375, 0.4025, 0.3975, 0.42875, 0.3775, 0.38625, 0.40625]
Start of Epoch 2
Average Train Loss: [2.0775, 2.0691, 2.0679, 2.0831, 2.0905, 2.0834, 2.0583, 2.0842, 2.0925, 2.0588]
Test Accuracy: [0.42625, 0.4525, 0.44625, 0.44125, 0.43625, 